## Exercise 09 : Plotly ##

## Imports ##

In [1]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

## Task #
Matplotlib и Seaborn — действительно мощные библиотеки, и с их помощью можно решать большинство задач по визуализации данных. Но они не предлагают возможность создавать интерактивные графики и анимации. В этом вам поможет Plotly. В этом упражнении вам нужно будет создать почти такой же график, как и в предыдущем, но в виде анимации.

Сделайте всё, что нужно, чтобы создать график, похожий на Readme.

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
df = pd.read_sql('SELECT * FROM checker WHERE uid LIKE "user%" AND labname="project1" AND status="ready"', conn, parse_dates=['timestamp']) 
df['date'] = df['timestamp'].dt.date
commit_counts = df.groupby(['uid', 'timestamp']).size().to_frame('commit_count').reset_index()
commit_counts = commit_counts.sort_values(['uid', 'timestamp'])

In [4]:
users = commit_counts['uid'].unique()
commit_counts['cum_commit_count'] = commit_counts.groupby('uid')['commit_count'].cumsum()

In [5]:
num_users = len(users)
colors = [
    f'hsl({h}, 70%, 50%)'
    for h in np.linspace(0, 360, num_users, endpoint=False)
]

# Размеры графика
height = 8
width = height * 1.5

# Создаем фигуру
fig = go.Figure()

for idx, user in enumerate(users):
    user_data = commit_counts[commit_counts['uid'] == user]
    user_data = user_data.sort_values('timestamp')
    fig.add_trace(
        go.Scatter(
            x=user_data['timestamp'],
            y=user_data['cum_commit_count'],
            mode='lines+markers',
            name=user,
            line=dict(width=3, color=colors[idx]),
            marker=dict(size=6, color=colors[idx]),
        )
    )

# Обновляем макет
fig.update_layout(
    title='Dynamic of commits per user in project1',
    width=int(width * 100),
    height=int(height * 100),
    xaxis_title='timestamp',
    yaxis_title='numTrials',
    legend=dict(
        x=1.02,
        y=0.5,
        xanchor='left',
        yanchor='middle',
        font=dict(size=10)
    ),
    margin=dict(r=20, t=50),
)

# Устанавливаем лимиты по х
min_date = commit_counts['timestamp'].min()
max_date = commit_counts['timestamp'].max()
fig.update_xaxes(range=[min_date, max_date])

# Показываем
fig.show()

In [6]:
conn.close()